<a href="https://colab.research.google.com/github/Shubh-3012/Wildlife-Image-Classification/blob/main/Wildlife_Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import random

from sklearn.metrics import confusion_matrix

from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras.datasets import fashion_mnist
from keras.utils import to_categorical

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_features = pd.read_csv("/content/drive/MyDrive/GB657_Notebooks/Final Project/Competition Files/train_features.csv", index_col="id")
test_features = pd.read_csv("/content/drive/MyDrive/GB657_Notebooks/Final Project/Competition Files/test_features.csv", index_col="id")
train_labels = pd.read_csv("/content/drive/MyDrive/GB657_Notebooks/Final Project/Competition Files/train_labels.csv", index_col="id")

In [ ]:
train_features.head()

,filepath,site
id,,
ZJ000000,train_features/ZJ000000.jpg,S0120
ZJ000001,train_features/ZJ000001.jpg,S0069
ZJ000002,train_features/ZJ000002.jpg,S0009
ZJ000003,train_features/ZJ000003.jpg,S0008
ZJ000004,train_features/ZJ000004.jpg,S0036


In [ ]:
test_features.head()

,filepath,site
id,,
ZJ016488,test_features/ZJ016488.jpg,S0082
ZJ016489,test_features/ZJ016489.jpg,S0040
ZJ016490,test_features/ZJ016490.jpg,S0040
ZJ016491,test_features/ZJ016491.jpg,S0041
ZJ016492,test_features/ZJ016492.jpg,S0040


In [ ]:
train_labels.head()

,antelope_duiker,bird,blank,civet_genet,hog,leopard,monkey_prosimian,rodent
id,,,,,,,,
ZJ000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
ZJ000001,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
ZJ000002,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
ZJ000003,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
ZJ000004,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [ ]:
species_labels = sorted(train_labels.columns.unique())
species_labels

['antelope_duiker',
 'bird',
 'blank',
 'civet_genet',
 'hog',
 'leopard',
 'monkey_prosimian',
 'rodent']

## Load Pre-trained EfficientNet Model

Load a pre-trained EfficientNet model (e.g., EfficientNet-B0 or B1 for a good balance of accuracy and speed) from `torchvision.models`. We'll adapt its final classification layer to output probabilities for our 8 species categories.


These are the necessary steps: import models, load efficientnet_b0 with pre-trained weights, get the input features of the original classifier, and replace it with a new linear layer suitable for 8 classes.



In [ ]:
import torchvision.models as models
import torch.nn as nn

num_classes = len(species_labels) # This is 8

# 1. Load a pre-trained EfficientNet model (e.g., EfficientNet-B0)
efficientnet_model = models.efficientnet_b0(pretrained=True)

# 2. Access the classifier layer and determine input features
# For EfficientNet, the classifier is a Sequential block, and the final linear layer is usually the second element.
in_features = efficientnet_model.classifier[1].in_features

# 3. Replace the final classification layer
efficientnet_model.classifier[1] = nn.Linear(in_features, num_classes)

print(efficientnet_model)
print(f"EfficientNet model loaded and classifier adapted for {num_classes} classes.")

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth


100%|██████████| 20.5M/20.5M [00:00<00:00, 127MB/s] 


EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

## Define EfficientNet Loss and Optimizer

Set up the appropriate loss function (nn.CrossEntropyLoss()) and an optimizer (like optim.Adam with a suitable learning rate) specifically for fine-tuning the pre-trained EfficientNet model on our dataset. This is crucial for adapting the model's learned features to our specific classification task.


In [ ]:
import torch.optim as optim

# 1. Define the loss function for the EfficientNet model
efficientnet_criterion = nn.CrossEntropyLoss()

# 2. Define the optimizer for the EfficientNet model
efficientnet_optimizer = optim.Adam(efficientnet_model.parameters(), lr=0.001)

# 3. Print a confirmation message
print("EfficientNet loss function and optimizer defined successfully.")

EfficientNet loss function and optimizer defined successfully.


## REVISED (Corrected) Model


# Correction made:
I modified the training loop for the EfficientNet model to correctly calculate the loss by converting one-hot encoded labels to class indices using `torch.argmax`. Then, I retrain the model and re-evaluate its accuracy.

Adjust the training loop in the EfficientNet model to convert one-hot encoded `labels` to class indices using `torch.argmax` before passing them to `efficientnet_criterion`. This will ensure the loss function receives the target in the expected format.

## Modify Training Loop for Correct Loss Calculation



In [ ]:
import torch
import os
from sklearn.model_selection import train_test_split
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image # Already imported in the first cell, but good to ensure it's available.

# Define the base path for your image files
LOCAL_BASE_PATH = "/content/drive/MyDrive/GB657_Notebooks/Final Project/Competition Files/"

# Define the ImagesDataset class
class ImagesDataset(Dataset):
    def __init__(self, features_df, labels_df, base_path, transform=None):
        self.features_df = features_df
        self.labels_df = labels_df
        self.base_path = base_path
        self.transform = transform

    def __len__(self):
        return len(self.features_df)

    def __getitem__(self, idx):
        img_id = self.features_df.index[idx]
        img_path = os.path.join(self.base_path, self.features_df.loc[img_id, 'filepath'])
        image = Image.open(img_path).convert('RGB') # Convert to RGB to ensure 3 channels
        label = torch.tensor(self.labels_df.loc[img_id].values, dtype=torch.float32)

        if self.transform:
            image = self.transform(image)

        return {"image": image, "label": label}

# Define image transformations
# EfficientNet-B0 typically expects 224x224 input
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),         # Convert PIL Image to PyTorch Tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # Normalize as expected by pre-trained models
])

# Split data into training and evaluation sets
x_train, x_eval, y_train, y_eval = train_test_split(
    train_features, train_labels,
    test_size=0.2, # 20% for evaluation
    random_state=42 # for reproducibility
)

# Create Dataset instances
train_dataset = ImagesDataset(x_train, y_train, base_path=LOCAL_BASE_PATH, transform=transform)
eval_dataset = ImagesDataset(x_eval, y_eval, base_path=LOCAL_BASE_PATH, transform=transform)

# Create DataLoader instances
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
eval_dataloader = DataLoader(eval_dataset, batch_size=32, shuffle=False)


# 1. Define the number of epochs for training
num_epochs_efficientnet = 5 # Starting with 5 epochs as suggested for computational reasons

# 2. Initialize an empty dictionary to store loss values
tracking_loss_efficientnet = {}

# 3. Check for GPU availability and move the model to the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
efficientnet_model.to(device)

print(f"Starting training for EfficientNet Model for {num_epochs_efficientnet} epochs on {device}")

# 4. Start a training loop iterating for the specified number of epochs
for epoch in range(1, num_epochs_efficientnet + 1):
    # 5. Print the current epoch number
    print(f"Starting epoch {epoch}")

    # Set model to training mode
    efficientnet_model.train()

    # 6. Iterate through the train_dataloader to get batches
    for batch_n, batch in tqdm(enumerate(train_dataloader), total=len(train_dataloader), desc=f"Epoch {epoch}"):
        # 7. Move the images and labels to the GPU if available
        images = batch["image"].to(device)
        labels = batch["label"].to(device)

        # 8. Zero out the gradients of the efficientnet_optimizer
        efficientnet_optimizer.zero_grad()

        # 9. Perform a forward pass
        outputs = efficientnet_model(images)

        # 10. Calculate the loss, converting one-hot labels to class indices
        loss = efficientnet_criterion(outputs, torch.argmax(labels, 1))

        # 11. Store the calculated loss
        tracking_loss_efficientnet[(epoch, batch_n)] = float(loss)

        # 12. Perform backpropagation
        loss.backward()

        # 13. Update the model's parameters
        efficientnet_optimizer.step()

# 14. After the training loop finishes, print a message
print("Training complete for EfficientNet Model.")


Starting training for EfficientNet Model for 5 epochs on cpu
Starting epoch 1


Epoch 1:   0%|          | 0/413 [00:00<?, ?it/s]/tmp/ipython-input-2907844180.py:93: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at /pytorch/torch/csrc/autograd/generated/python_variable_methods.cpp:836.)
  tracking_loss_efficientnet[(epoch, batch_n)] = float(loss)
Epoch 1: 100%|██████████| 413/413 [1:53:23<00:00, 16.47s/it]


Starting epoch 2


Epoch 2: 100%|██████████| 413/413 [57:07<00:00,  8.30s/it]


Starting epoch 3


...

Starting epoch 4


Epoch 4: 100%|██████████| 413/413 [55:25<00:00,  8.05s/it]


Starting epoch 5


Epoch 5: 100%|██████████| 413/413 [55:43<00:00,  8.10s/it]

Training complete for EfficientNet Model.


## Evaluate EfficientNet Model

In [ ]:
from sklearn.metrics import accuracy_score

efficientnet_model.eval() # Set the model to evaluation mode

eval_dataset = ImagesDataset(x_eval, y_eval, base_path=LOCAL_BASE_PATH)
eval_dataloader = DataLoader(eval_dataset, batch_size=32)

all_labels_eval = []
all_predictions_eval = []

with torch.no_grad(): # Disable gradient calculations for inference
    for batch in tqdm(eval_dataloader, desc="Evaluating EfficientNet on Eval Data"):
        images = batch["image"].to(device)
        labels = batch["label"].to(device)

        outputs = efficientnet_model(images)

        # Convert one-hot encoded labels to class indices
        _, predicted = torch.max(outputs.data, 1)
        _, true_labels = torch.max(labels.data, 1)

        all_labels_eval.extend(true_labels.cpu().numpy())
        all_predictions_eval.extend(predicted.cpu().numpy())

accuracy_efficientnet_eval = accuracy_score(all_labels_eval, all_predictions_eval)
print(f"Accuracy of EfficientNet on the evaluation dataset: {accuracy_efficientnet_eval:.4f}")

efficientnet_model.train() # Set the model back to training mode if you plan to train more

Evaluating EfficientNet on Eval Data: 100%|██████████| 65/65 [02:16<00:00,  2.09s/it]

Accuracy of EfficientNet on the evaluation dataset: 0.8142


EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat